In [ ]:
import tkinter as tk
from tkinter import filedialog, messagebox, ttk
from datetime import datetime
import serial
import time
import pyvisa
import csv
import os
import subprocess
import random

# ------------------- Configuraciones iniciales -------------------
bandas = [1000, 1250]
rating = [0.185, 0.325, 0.530, 0.770, 0.890, 0.915, 0.945, 0.970, 1.000,
          1.025, 1.055, 1.085, 1.120, 1.290, 1.875, 3.045, 5.380]
spl = 90.0
tolerancia = 1
modo_simulacion = False
ser = None
rigol = None
ventana_resultados = None

resultados_globales = []

# ------------------- Funciones del sistema -------------------
def configurar_sonometro(port='/dev/cu.usbserial-0001', baudrate=9600, timeout=1):
    if modo_simulacion:
        print("🔧 MODO SIMULACIÓN: Simulando conexión con sonómetro.")
        return "simulado"
    try:
        return serial.Serial(port, baudrate, timeout=timeout)
    except serial.SerialException as e:
        print(f"Error al conectar con el sonómetro: {e}")
        return None

def prender_sonometro(ser):
    if modo_simulacion:
        print("🔧 MODO SIMULACIÓN: Simulando encendido de sonómetro.")
        return
    comando = bytes.fromhex('02 01 43 53 54 41 31 03 34 0D 0A')
    ser.write(comando)
    time.sleep(1)
    ser.readline()

def obtener_niveles_tercio_octava(ser):
    if modo_simulacion:
        print("🔧 MODO SIMULACIÓN: Simulando niveles de tercio de octava.")
        return {banda: str(round(random.uniform(85, 95), 2)) for banda in bandas}
    comando = bytes.fromhex('02 01 43 44 54 54 31 20 3F 03 00 0D 0A')
    ser.write(comando)
    time.sleep(1)
    response = ser.readline().decode('utf-8', errors='ignore').strip()
    niveles1 = response.split(",")[27:29]
    return dict(zip(bandas, niveles1))

def configurar_fuente():
    if modo_simulacion:
        print("🔧 MODO SIMULACIÓN: Simulando fuente Rigol.")
        return "rigol_simulado"
    try:
        rm = pyvisa.ResourceManager('@py')
        dispositivos = rm.list_resources()
        id = [dev for dev in dispositivos if "USB" in dev]
        rigol = rm.open_resource(id[0])
        rigol.timeout = 10000
        rigol.write("*RST")
        time.sleep(0.5)
        print("Fuente detectada y reiniciada")
        return rigol
    except Exception as e:
        print(f"Error al configurar la fuente: {e}")
        return None

def prender_fuente(rigol):
    if modo_simulacion:
        print("🔧 MODO SIMULACIÓN: Fuente encendida (simulada).")
        return
    rigol.write("*CLS")
    time.sleep(0.5)
    rigol.write("SOUR1:FUNC SEN")
    time.sleep(0.5)
    rigol.write("SOUR1:VOLT 0.001")
    time.sleep(0.5)
    rigol.write("SOUR1:VOLT:OFFS 0")
    time.sleep(0.5)
    rigol.write("OUTP1 ON")
    time.sleep(0.5)

def ajustar_fuente():
    global ser, rigol
    rigol = configurar_fuente()
    if rigol:
        prender_fuente(rigol)
        ser = configurar_sonometro()
        if ser:
            prender_sonometro(ser)
            niveles = obtener_niveles_tercio_octava(ser)
            for banda in bandas:
                nivel_actual = float(niveles.get(banda, 0))
                ajustar_voltaje_rigol(rigol, ser, banda, nivel_actual)
            messagebox.showinfo("Ajuste completado", f"Fuente ajustada exitosamente. Nivel SPL final: {nivel_actual:.2f} dB")

def ajustar_voltaje_rigol(rigol, ser, banda, nivel_actual):
    volt = 0.05
    iteraciones = 0
    while abs(nivel_actual - spl) > tolerancia and iteraciones < 20:
        if not modo_simulacion:
            rigol.write(f"SOUR1:VOLT {volt:.4f}")
        time.sleep(1)
        niveles = obtener_niveles_tercio_octava(ser)
        nivel_actual = float(niveles.get(banda, nivel_actual))
        diferencia_db = spl - nivel_actual
        volt *= 10 ** (diferencia_db / 20)
        if volt <= 0 or volt > 5:
            break
        iteraciones += 1

def mostrar_resultados_en_tabla():
    global resultados_globales, ventana_resultados
    if ventana_resultados:
        ventana_resultados.destroy()

    ventana_resultados = tk.Toplevel()
    ventana_resultados.title("Resultados de Calibración por Banda")

    contenedor_scroll = tk.Canvas(ventana_resultados)
    contenedor_scroll.pack(side="left", fill="both", expand=True)

    scrollbar = ttk.Scrollbar(ventana_resultados, orient="horizontal", command=contenedor_scroll.xview)
    scrollbar.pack(side="bottom", fill="x")

    contenedor_scroll.configure(xscrollcommand=scrollbar.set)

    frame_tablas = tk.Frame(contenedor_scroll)
    contenedor_scroll.create_window((0, 0), window=frame_tablas, anchor="nw")

    resultados_por_banda = {banda: [] for banda in bandas}

    banda_idx = 0
    for fila in resultados_globales:
        if banda_idx < len(bandas):
            resultados_por_banda[bandas[banda_idx]].append(fila)
            if len(resultados_por_banda[bandas[banda_idx]]) == 17:
                banda_idx += 1

    for idx, banda in enumerate(bandas):
        tk.Label(frame_tablas, text=f"Frecuencia central: {banda} Hz", font=("Helvetica", 12, "bold"), bg="yellow").grid(row=0, column=idx, columnspan=4, sticky="ew")

        tabla = ttk.Treeview(frame_tablas, columns=("frecuencia", "nivel1", "nivel2", "nivel3"), show="headings", height=20)
        tabla.heading("frecuencia", text="Frecuencia (Hz)")
        tabla.heading("nivel1", text="Nivel 1 (dB)")
        tabla.heading("nivel2", text="Nivel 2 (dB)")
        tabla.heading("nivel3", text="Nivel 3 (dB)")

        tabla.grid(row=1, column=idx, padx=10, pady=10)

        for fila in resultados_por_banda[banda]:
            tabla.insert("", "end", values=fila)

    frame_tablas.update_idletasks()
    contenedor_scroll.configure(scrollregion=contenedor_scroll.bbox("all")))

def iniciar_calibracion():
    global ser, rigol, resultados_globales
    resultados_globales.clear()
    progreso = ttk.Progressbar(ventana, mode="determinate", maximum=len(bandas), length=400)
    progreso.pack(pady=10)

    if not rigol:
        rigol = configurar_fuente()
        prender_fuente(rigol)
    if not ser:
        ser = configurar_sonometro()
        prender_sonometro(ser)

    niveles = obtener_niveles_tercio_octava(ser)
    with open("datos_sonometro.csv", "w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(["Frecuencia (Hz)", "Nivel 1 (dB)", "Nivel 2 (dB)", "Nivel 3 (dB)"])
        for idx, banda in enumerate(bandas):
            writer.writerow([f"Frecuencia central: {banda} Hz"])
            ajustar_voltaje_rigol(rigol, ser, banda, float(niveles.get(banda, 0)))
            resultados = []
            for i in range(min(9, len(rating)) - 1, -1, -1):
                frecuencia = round(banda * rating[i], 3)
                time.sleep(0.5)
                niveles_medidos = [float(obtener_niveles_tercio_octava(ser).get(banda, 0)) for _ in range(3)]
                resultados.append((frecuencia, *niveles_medidos))
            for i in range(9, min(17, len(rating))):
                frecuencia = round(banda * rating[i], 3)
                time.sleep(0.5)
                niveles_medidos = [float(obtener_niveles_tercio_octava(ser).get(banda, 0)) for _ in range(3)]
                resultados.append((frecuencia, *niveles_medidos))
            resultados.sort(key=lambda x: x[0])
            for fila in resultados:
                resultados_globales.append(fila)
                writer.writerow(fila)
            progreso["value"] = idx + 1
            ventana.update_idletasks()
        if not modo_simulacion:
            rigol.write("OUTP1 OFF")
    progreso.destroy()
    messagebox.showinfo("Calibración finalizada", "Proceso de calibración completado exitosamente.")

def descargar_y_abrir_csv():
    ruta = filedialog.asksaveasfilename(defaultextension=".csv", filetypes=[("Archivo CSV", "*.csv")])
    if ruta:
        with open("datos_sonometro.csv", "r") as original:
            contenido = original.read()
        with open(ruta, "w") as nuevo:
            nuevo.write(contenido)
        subprocess.Popen(["notepad.exe", ruta], shell=True)

# -------------------- Interfaz Gráfica --------------------
ventana = tk.Tk()
ventana.title("Sistema Calibración filtros de banda")
ventana.geometry("1920x1080")

titulo = tk.Label(ventana, text="Sistema Calibración filtros de banda", font=("Helvetica", 24, "bold"))
titulo.pack(pady=20)

frame_formulario = tk.Frame(ventana)
frame_formulario.pack(pady=10)

campos = [
    ("Empresa:", "entry_empresa"),
    ("Modelo del Sonómetro:", "entry_modelo"),
    ("Serial del Sonómetro:", "entry_serial"),
    ("Fecha de Calibración:", "entry_fecha"),
    ("Responsable:", "entry_operador"),
    ("Humedad Relativa (%):", "entry_humedad"),
    ("Temperatura (°C):", "entry_temperatura")
]

entradas = {}
for i, (label_text, var_name) in enumerate(campos):
    tk.Label(frame_formulario, text=label_text, font=("Helvetica", 14)).grid(row=i, column=0, sticky="e", padx=10, pady=10)
    entry = tk.Entry(frame_formulario, font=("Helvetica", 14), width=40)
    entry.grid(row=i, column=1, padx=10, pady=10)
    if "fecha" in var_name:
        entry.insert(0, datetime.now().strftime("%Y-%m-%d"))
    entradas[var_name] = entry

frame_botones = tk.Frame(ventana)
frame_botones.pack(pady=30)

tk.Button(frame_botones, text="Ajustar Fuente", font=("Helvetica", 16), width=20, bg="orange", fg="white", command=ajustar_fuente).grid(row=0, column=0, padx=30)
tk.Button(frame_botones, text="Iniciar Calibración", font=("Helvetica", 16), width=20, bg="green", fg="white", command=iniciar_calibracion).grid(row=0, column=1, padx=30)
tk.Button(frame_botones, text="Descargar y Abrir CSV", font=("Helvetica", 16), width=25, bg="blue", fg="white", command=descargar_y_abrir_csv).grid(row=0, column=2, padx=30)
tk.Button(frame_botones, text="Ver Resultados", font=("Helvetica", 16), width=20, bg="purple", fg="white", command=mostrar_resultados_en_tabla).grid(row=0, column=3, padx=30)

ventana.mainloop()
